<a href="https://colab.research.google.com/github/fenwickslab/fenwicks/blob/master/tutorials/tutorial9_gpu_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu May 23 16:40:46 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    17W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip install -qq -U cufflinks

     |████████████████████████████████| 81kB 1.9MB/s 


In [0]:
import tensorflow as tf
import os
import numpy as np

In [4]:
if tf.io.gfile.exists('./fenwicks'):
  tf.io.gfile.rmtree('./fenwicks')
!git clone https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw

Cloning into 'fenwicks'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 1692 (delta 4), reused 15 (delta 4), pack-reused 1674
Receiving objects: 100% (1692/1692), 43.14 MiB | 12.47 MiB/s, done.
Resolving deltas: 100% (1035/1035), done.


In [5]:
PROJECT = 'tutorial1'
ROOT_DIR = '.'
BATCH_SIZE = 256 #@param ["64", "128", "256", "512"] {type:"raw"}
EPOCHS = 24 #@param {type:"slider", min:0, max:100, step:1}

In [6]:
_, work_dir = fw.io.get_project_dirs(ROOT_DIR, PROJECT)

In [7]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
n_train, n_test = len(X_train), len(X_test)
X_train = (X_train.reshape(-1, 28, 28, 1) / 255.0).astype('float32')
X_test = (X_test.reshape(-1, 28, 28, 1) / 255.0).astype('float32')
y_train = y_train.astype('int64')
y_test = y_test.astype('int64')
n_classes = np.max(y_train)+1

11493376/11490434 [==============================] - 0s 0us/step


In [8]:
fw.plt.plot_counts_pie(y_train)

In [9]:
train_input_func = lambda params: fw.data.numpy_ds(X_train, y_train, batch_size=params['batch_size'], 
                                                   shuffle_buf_sz=n_train, training=True)
eval_input_func = lambda params: fw.data.numpy_ds(X_test, y_test, batch_size=params['batch_size'], training=False)

In [10]:
fw.anim.show_input_func(eval_input_func, converter=fw.image.gray2rgb)

In [11]:
def build_nn(c=6, c_dense=200):
  model = fw.Sequential()
  model.add(fw.layers.ConvBN(c, kernel_size=3))
  model.add(fw.layers.ConvBN(c*2, kernel_size=6, strides=2))
  model.add(fw.layers.ConvBN(c*4, kernel_size=6, strides=2))
  model.add(tf.layers.Flatten())
  model.add(fw.layers.DenseBN(c_dense, drop_rate=0.5))
  model.add(fw.layers.Classifier(n_classes))
  return model

In [12]:
steps_per_epoch = n_train // BATCH_SIZE
total_steps = steps_per_epoch * EPOCHS

lr_func = fw.train.exp_decay_lr(base_lr=0.0001, init_lr=0.01, decay_steps=2000)
opt_func = fw.train.adam_optimizer(lr_func)
model_func = fw.train.get_clf_model_func(build_nn, opt_func)

In [13]:
fw.plt.plot_lr_func(lr_func, total_steps)

In [14]:
est = fw.train.get_tpu_estimator(steps_per_epoch, model_func, work_dir, trn_bs=BATCH_SIZE, val_bs=n_test)
est.train(train_input_func, steps=total_steps)

INFO:tensorflow:Using config: {'_model_dir': './work/tutorial1/2019-05-23-16:41:21', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f38707d2208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=234, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infee

The evaluation has only 1 step, since its batch size is the same as the number of validation records.

In [15]:
result = est.evaluate(eval_input_func, steps=1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running eval on CPU
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-23T16:42:57Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./work/tutorial1/2019-05-23-16:41:21/model.ckpt-5616
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2019-05-23-16:42:58
INFO:tensorflow:Saving dict for global step 5616: accuracy = 0.9941, global_step = 5616, loss = 0.026700381
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5616: ./work/tutorial1/2019-05-23-16:41:21/model.ckpt-5616
INFO:tensorflow:evaluation_loop marked as finished


In [16]:
print(f'Test results: accuracy={result["accuracy"] * 100: .2f}%, loss={result["loss"]: .2f}.')

Test results: accuracy= 99.41%, loss= 0.03.


In [17]:
fw.io.create_clean_dir(work_dir)